# Set partitioning using DMQ

# Partitioning to more than two sets

The goal is to partition a set of numbers to multiple sets of equal size. A decision variable that serves this purpose is a binary variable $x_{i,j}$ that assigns the value $i$ to the set $j$ if $x_{i,j}=1$. This is equivalent to using a discrete variable $x_i$ that takes the value $x_i = j$. Using such a discrete variable, gaurantees that only one of the sets is selected.

**note**: The two binary ($x_{i,j}=1$) and discerete ($x_i = j$) can be used interchangeably. For most mathematical expressions it is more convenient to use the binary form.

In [2]:
from dimod import DiscreteQuadraticModel

values = [7, 2, 3, 1, 8, 3, 1, 2, 9]
dqm = DiscreteQuadraticModel()
n = len(values)
m = 3 # num_partitions

x = {i: dqm.add_variable(m) for i in range(n)}

# No objective, only constraints

For each pair of sets, ensure that the sum of the values in both sets is the same

$$ \sum_i v_i x_{ij} = \sum_i v_i x_{ik} $$ for all $j$ and $k$

Or equivalently:

$$ \sum_i v_i x_{ij} - \sum_i v_i x_{ik} = 0$$

In [4]:
from itertools import combinations
for k, l in combinations(range(m), r=2):
    dqm.add_linear_equality_constraint(
    [(x[i], k, values[i]) for i in range(n)] + [(x[i], l, -values[i]) for i in range(n)],
    constant=0,
    lagrange_multiplier=10)

Add a constraint to make sure each value is assign to exactly one set. 

In [100]:
# Actually, no need! This constraint is a built-in when using discrete variables.

Solve using one of the solvers. You may have to run it a few times.

In [6]:
from dimod import ExactDQMSolver

res = ExactDQMSolver().sample_dqm(dqm).truncate(5)
print(res)

  0 1 2 3 4 5 6 7 8 energy num_oc.
0 0 0 2 1 2 0 2 1 1    0.0       1
1 2 1 0 1 0 2 0 2 1    0.0       1
2 0 0 1 1 1 0 2 2 2    0.0       1
3 0 1 2 1 2 0 2 0 1    0.0       1
4 2 2 0 1 0 2 0 1 1    0.0       1
['DISCRETE', 5 rows, 5 samples, 9 variables]


# Result

In [11]:
sample = res.first.sample

print(sample)
print(sum(values))
for k in range(m):
    set1 = [values[i] for i in x if sample[x[i]] == k]
    print(sum(set1), set1)

{0: 0, 1: 0, 2: 2, 3: 1, 4: 2, 5: 0, 6: 2, 7: 1, 8: 1}
36
12 [7, 2, 3]
12 [1, 2, 9]
12 [3, 8, 1]
